# Tratar Dados

# Bibliotecas

In [43]:
import pandas as pd

# Tratamento de dados dicionário

## Importar Dicionários

In [44]:
def abrir_dics(path, sheet_name='dicionário pnad covid'):

    # Lê arquivo Excel no caminho especificado (path)
    prepdics = pd.read_excel(path, sheet_name=sheet_name, header=1, usecols='B:F')

    # Remove a coluna 'Quesito' e as primeiras 3 linhas
    prepdics = prepdics.iloc[3:].drop('Quesito', axis=1)

    # Renomeia as colunas
    prepdics.columns = ['variavel', 'questao', 'tipo', 'descricao']

    # Reset do índice
    prepdics = prepdics.reset_index(drop=True)

    return prepdics

dic09 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_092020_20220621.xls')
dic10 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_102020_20220621.xls')
dic11 = abrir_dics('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/Dicionario_PNAD_COVID_112020_20220621.xls')

dics = [dic09, dic10, dic11]

Visualizar primeiras alterações feitas, dos 3 dicionarios

In [45]:
print("dic09:")
print(dic09.head(5))
print(dic09.shape)

print("\ndic10:")
print(dic10.head(5))
print(dic10.shape)

print("\ndic11:")
print(dic11.head(5))
print(dic11.shape)

dic09:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  Unidade da Federação   11  Rondônia
1      NaN                   NaN   12      Acre
2      NaN                   NaN   13  Amazonas
3      NaN                   NaN   14   Roraima
4      NaN                   NaN   15      Pará
(656, 4)


## Tirar valores null e preencher campos

In [46]:
def limpar_dicionarios(*dfs):
    # Lista para armazenar os dataframes limpos
    clean_dfs = []
    for df in dfs:
        # Preenche os valores NaN e faz uma cópia
        clean = df.ffill().copy()
        # Converte para minúsculas as colunas de string, exceto a coluna 'tipo'
        clean = clean.apply(lambda x: x.str.lower() if x.name != 'tipo' and x.name != 'variavel' and x.dtype == 'object' else x)
        # Substitui padrões nas colunas de string
        replaces = {'  ': ' ', r'\n': ' ', r'\r': ' ', r'/': ' ou ', r'%': '_por_cento'}
        clean.replace(replaces, regex=True, inplace=True)
        # Adiciona o dataframe limpo à lista
        clean_dfs.append(clean)
    return clean_dfs

clean_dfs = limpar_dicionarios(dic09, dic10, dic11)
dic09_clean, dic10_clean, dic11_clean = clean_dfs

dic_clean = [dic09_clean, dic10_clean, dic11_clean]


In [47]:
print("dic09:")
print(dic09_clean.head(5))
print(dic09_clean.shape)

print("\ndic10:")
print(dic10_clean.head(5))
print(dic10_clean.shape)

print("\ndic11:")
print(dic11_clean.head(5))
print(dic11_clean.shape)

dic09:
  variavel               questao tipo descricao
0       UF  unidade da federação   11  rondônia
1       UF  unidade da federação   12      acre
2       UF  unidade da federação   13  amazonas
3       UF  unidade da federação   14   roraima
4       UF  unidade da federação   15      pará
(641, 4)

dic10:
  variavel               questao tipo descricao
0       UF  unidade da federação   11  rondônia
1       UF  unidade da federação   12      acre
2       UF  unidade da federação   13  amazonas
3       UF  unidade da federação   14   roraima
4       UF  unidade da federação   15      pará
(641, 4)

dic11:
  variavel               questao tipo descricao
0       UF  unidade da federação   11  rondônia
1       UF  unidade da federação   12      acre
2       UF  unidade da federação   13  amazonas
3       UF  unidade da federação   14   roraima
4       UF  unidade da federação   15      pará
(656, 4)


## Concatena os dicionários

In [48]:
def processar_dicionarios(dic_clean, drop_duplicates=True):
    # Concatena todos os dicionários
    df_dics = pd.concat(dic_clean)
    # Preenche os valores NaN
    df_dics = df_dics.ffill()
    # Remove duplicatas
    if drop_duplicates:
        df_dics = df_dics.drop_duplicates()
    # Remove duplicatas com base em 'variavel' e 'descricao'
    df_dics = df_dics.drop_duplicates(subset=['variavel', 'descricao'])

    return df_dics

# Usando a função
df_dics_processed = processar_dicionarios(dic_clean, drop_duplicates=True)
print(df_dics_processed.head(5), "\nshape:", df_dics_processed.shape)

  variavel               questao tipo descricao
0       UF  unidade da federação   11  rondônia
1       UF  unidade da federação   12      acre
2       UF  unidade da federação   13  amazonas
3       UF  unidade da federação   14   roraima
4       UF  unidade da federação   15      pará 
shape: (650, 4)


## Visualizar as questões

In [50]:
def questions (df, variavel=None, key='tipo', value='questao'):
    if variavel is None:
        return df.set_index(key)[value].to_dict()
    else:
        return df.query(f'variavel == "{variavel}"').set_index(key)[value].to_dict()

# Visualizar todas as questões
questionsdic = questions(df_dics_processed, key='variavel', value='questao')
questionsdic

{'UF': 'unidade da federação',
 'CAPITAL': 'capital',
 'RM_RIDE': 'região metropolitana e região administrativa integrada de desenvolvimento',
 'V1008': 'número de seleção do domicílio',
 'V1012': 'semana no mês',
 'V1013': 'mês da pesquisa',
 'V1016': 'número da entrevista no domicílio',
 'Estrato': 'estrato',
 'UPA': 'upa',
 'V1022': 'situação do domicílio',
 'V1023': 'tipo de área',
 'V1030': 'projeção da população',
 'V1031': 'peso do domicílio e das pessoas',
 'V1032': 'peso do domicílio e das pessoas',
 'posest': 'domínios de projeção',
 'A001': 'número de ordem',
 'A001A': 'condição no domicílio',
 'A001B1': 'dia de nascimento',
 'A001B2': 'mês de nascimento',
 'A001B3': 'ano de nascimento',
 'A002': 'idade do morador ',
 'A003': 'sexo',
 'A004': 'cor ou raça',
 'A005': 'escolaridade',
 'A006': 'frequenta escola',
 'A007': 'na semana passada, _____ foram disponibilizadas atividades escolares para realizar em casa?',
 'A008': 'na semana passada, em quantos dias _____ dedicou-se à

## Criar excel do dicionário

In [49]:
# df_dics_processed.to_excel("/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/dicionario_tratado.xlsx")

# Tratamento de dados PNAD COVID

3 meses para construção da solução

In [51]:
df09 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_092020.csv')
df10 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_102020.csv')
df11 = pd.read_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/PNAD_COVID_112020.csv')

covid_dfs = [df09, df10, df11]

for d in covid_dfs:
    print(f'shape: {d.shape}')

shape: (387298, 145)
shape: (380461, 145)
shape: (381438, 148)


## Compara as colunas

In [52]:
def comparar_colunas(df1, df2):
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)

    cols_only_in_df1 = cols1 - cols2
    cols_only_in_df2 = cols2 - cols1

    if cols_only_in_df1:
        print(f"Colunas presentes apenas em df1: {cols_only_in_df1}")
    if cols_only_in_df2:
        print(f"Colunas presentes apenas em df2: {cols_only_in_df2}")

# Comparar todos os pares de DataFrames
comparar_par = [(df09, df10), (df09, df11), (df10, df11)]

for df1, df2 in comparar_par:
    comparar_colunas(df1, df2)

Colunas presentes apenas em df2: {'A006A', 'A007A', 'A006B'}
Colunas presentes apenas em df2: {'A006A', 'A007A', 'A006B'}


## Concatenar

In [53]:
df_concat = pd.concat([df09, df10, df11])
df_concat.shape

(1149197, 148)

## Selecionar as colunas

In [54]:
# Lista de variáveis selecionadas
variaveis_selecionadas = ['UF' ,'A002' ,'A003' ,'A004' ,'A005' ,'A006A' ,'A006' ,'F001' ,'V1022' ,'C01011' ,'B011' ,'B0011' ,'B0012'
                          ,'B0013' ,'B0014' ,'B0015' ,'B0016' ,'B0017' ,'B0018' ,'B0019' ,'B00110' ,'B00111' ,'B00112' ,'B00113' ,'B002'
                          ,'B0043' ,'B0046' ,'B0041' ,'B0044' ,'B0042' ,'B0045' ,'B008' ,'B009A' ,'B009B' ,'B009C' ,'B009D' ,'B009E'
                          ,'B009F' ,'B0101' ,'B0102' ,'B0103' ,'B0104' ,'B0105' ,'B0106' ,'C002' ,'C003' ,'C004' ,'C006' ,'C007'
                          ,'C008' ,'C013']

# Selecionando apenas as colunas desejadas
df_covid = df_concat[variaveis_selecionadas]

## Ajustado os dados usando do dicionário

In [55]:
def rename_codes(df, var_names):
    for var_name in var_names:
        # Crie um dicionário de códigos e descrições
        code_dict = df_dics_processed.query(f'variavel == "{var_name}"')[['tipo', 'descricao']].set_index('tipo')['descricao'].to_dict()

        # Renomeie a variável no DataFrame
        df.loc[:, var_name] = df[var_name].map(code_dict)

    return df


# Lista de variáveis para renomear
var_names_to_rename = ['UF', 'A003', 'A004', 'A005', 'F001', 'V1022',
                       'B0011', 'B0012', 'B0013', 'B0014','B0015','B0016', 'B0017',
                       'B0018', 'B0019', 'B00110','B00111', 'B00112', 'B00113', 'B008',
                       'B0101','B0102', 'B0103', 'B0104', 'B0105', 'B0106']


# Renomeie as variáveis no DataFrame
df_covid = rename_codes(df_covid, var_names_to_rename)

# Exiba as primeiras linhas do DataFrame
df_covid.head()

,UF,A002,A003,A004,A005,A006A,A006,F001,V1022,C01011,...,B0104,B0105,B0106,C002,C003,C004,C006,C007,C008,C013
0,rondônia,36,homem,parda,médio completo,NaN,NaN,próprio - já pago,urbana,4.0,...,não,não,não,NaN,NaN,NaN,2.0,4.0,48.0,NaN
1,rondônia,30,mulher,parda,superior completo,NaN,NaN,próprio - já pago,urbana,4.0,...,não,não,não,NaN,NaN,NaN,2.0,7.0,36.0,NaN
2,rondônia,13,homem,parda,fundamental incompleto,NaN,1.0,próprio - já pago,urbana,NaN,...,não,não,não,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rondônia,11,homem,parda,fundamental incompleto,NaN,1.0,próprio - já pago,urbana,NaN,...,não,não,não,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rondônia,57,mulher,branca,fundamental incompleto,NaN,NaN,próprio - já pago,urbana,NaN,...,não,não,não,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
columns_to_map = {
    'A006A': {1.0: 'pública', 2.0: 'privada'},
    'A006':{1.0: 'sim', 2.0: 'não'},
    'C002': {1.0: 'sim', 2.0: 'não'},
    'C006': {1.0: 'sim', 2.0: 'não'},
    'C013': {1.0: 'sim', 2.0: 'não'},
    'C004': {1.0: 'sim', 2.0: 'não', 3.0: 'não remunerado'},
    'B002': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0041': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0042': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0043': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0044': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0045': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B0046': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B009A': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B009C': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B009E': {1.0: 'sim', 2.0: 'não', 9.0: 'não aplicavel'},
    'B009B': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'não recebeu o resultado', 9.0: 'não aplicavel'},
    'B009D': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'não recebeu o resultado', 9.0: 'não aplicavel'},
    'B009F': {1.0: 'positivo', 2.0: 'negativo', 3.0: 'inconclusivo', 4.0: 'não recebeu o resultado', 9.0: 'não aplicavel'}
}

for col, mapping in columns_to_map.items():
    df_covid[col] = df_covid[col].map(mapping).fillna('não aplicavel')

<ipython-input-56-0f9129f78645>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid[col] = df_covid[col].map(mapping).fillna('não aplicavel')


In [57]:
# Mapeamento para a coluna C003 - tipo de trabalho
c003_mapping = {
    1.0:  'quarentena, ou férias coletivas',
    2.0:  'férias ou jornada variável ',
    3.0:  'maternidade ou paternidade',
    4.0:  'saúde ou acidente remunerada ',
    5.0:  'outro tipo de licença remunerada',
    6.0:  'afastamento sem ser remunerado',
    7.0:  'fatores ocasionais',
    8.0:  'outro motivo',
}

# Mapear a coluna C003
df_covid['C003'] = df_covid['C003'].map(c003_mapping).fillna('não aplicável')

<ipython-input-57-1391522f85ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C003'] = df_covid['C003'].map(c003_mapping).fillna('não aplicável')


In [58]:
# Mapeamento para a coluna C01011
c01011_mapping = {
    0.0: '0 - 100',
    1.0:  '101 - 300',
    2.0:  '301 - 600',
    3.0:  '601 - 800',
    4.0:  '801 - 1.600',
    5.0:  '1.601 - 3.000',
    6.0:  '3.001 - 10.000',
    7.0:  '10.001 - 50.000',
    8.0:  '50.001 - 100.000',
    9.0:  'mais de 100.000'
}

# Mapear a coluna C01011
df_covid['C01011'] = df_covid['C01011'].map(c01011_mapping).fillna('não aplicável')

<ipython-input-58-f7b4a60743e8>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C01011'] = df_covid['C01011'].map(c01011_mapping).fillna('não aplicável')


In [59]:
# Mapeamento para a coluna C007 - tipo de trabalho
c007_mapping = {
    1.0: 'trabalhador domestico',
    2.0: 'militar',
    3.0: 'policial ou bombeiro militar',
    4.0: 'empregado do setor privado',
    5.0: 'empregado do setor público',
    6.0: 'empregador',
    7.0: 'conta propria',
    8.0: 'trabalhador familiar',
    9.0: 'fora do mercado de trabalho'
}

# Mapear a coluna C007
df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('não aplicável')

<ipython-input-59-e929e23ca2c0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['C007'] = df_covid['C007'].map(c007_mapping).fillna('não aplicável')


In [60]:
# Mapeamento para a coluna B011 - restringiu_contato_fisico
b011_mapping = {
    1: 'sem restricao',
    2: 'reduziu o contato',
    3: 'so saiu para necessidade básica',
    4: 'ficou somente em casa',
    9: 'não aplicável'
}

# Mapear a coluna B011
df_covid['B011'] = df_covid['B011'].map(b011_mapping).fillna('não aplicável')

<ipython-input-60-f368f707cd01>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['B011'] = df_covid['B011'].map(b011_mapping).fillna('não aplicável')


## Verificar null

In [61]:
df_covid.isnull().sum()

UF             0
A002           0
A003           0
A004           0
A005           0
A006A          0
A006           0
F001           0
V1022          0
C01011         0
B011           0
B0011          0
B0012          0
B0013          0
B0014          0
B0015          0
B0016          0
B0017          0
B0018          0
B0019          0
B00110         0
B00111         0
B00112         0
B00113         0
B002           0
B0043          0
B0046          0
B0041          0
B0044          0
B0042          0
B0045          0
B008           0
B009A          0
B009B          0
B009C          0
B009D          0
B009E          0
B009F          0
B0101          0
B0102          0
B0103          0
B0104          0
B0105          0
B0106          0
C002           0
C003           0
C004           0
C006           0
C007           0
C008      713200
C013           0
dtype: int64

## Preencher os NaN

In [63]:
# NaN to 0
df_covid = df_covid.fillna(0)
df_covid.head()

,UF,A002,A003,A004,A005,A006A,A006,F001,V1022,C01011,...,B0104,B0105,B0106,C002,C003,C004,C006,C007,C008,C013
0,rondônia,36,homem,parda,médio completo,não aplicavel,não aplicavel,próprio - já pago,urbana,801 - 1.600,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não,empregado do setor privado,48.0,não aplicavel
1,rondônia,30,mulher,parda,superior completo,não aplicavel,não aplicavel,próprio - já pago,urbana,801 - 1.600,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não,conta propria,36.0,não aplicavel
2,rondônia,13,homem,parda,fundamental incompleto,não aplicavel,sim,próprio - já pago,urbana,não aplicável,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel
3,rondônia,11,homem,parda,fundamental incompleto,não aplicavel,sim,próprio - já pago,urbana,não aplicável,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel
4,rondônia,57,mulher,branca,fundamental incompleto,não aplicavel,não aplicavel,próprio - já pago,urbana,não aplicável,...,não,não,não,não,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel


In [64]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1149197 entries, 0 to 381437
Data columns (total 51 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   UF      1149197 non-null  object 
 1   A002    1149197 non-null  int64  
 2   A003    1149197 non-null  object 
 3   A004    1149197 non-null  object 
 4   A005    1149197 non-null  object 
 5   A006A   1149197 non-null  object 
 6   A006    1149197 non-null  object 
 7   F001    1149197 non-null  object 
 8   V1022   1149197 non-null  object 
 9   C01011  1149197 non-null  object 
 10  B011    1149197 non-null  object 
 11  B0011   1149197 non-null  object 
 12  B0012   1149197 non-null  object 
 13  B0013   1149197 non-null  object 
 14  B0014   1149197 non-null  object 
 15  B0015   1149197 non-null  object 
 16  B0016   1149197 non-null  object 
 17  B0017   1149197 non-null  object 
 18  B0018   1149197 non-null  object 
 19  B0019   1149197 non-null  object 
 20  B00110  1149197 non-null  obje

## Renomear as colunas

In [65]:
# Dicionário com os novos nomes das variáveis
df_covid = df_covid.rename(columns={
    # Social
    'UF': 'uf',
    'A002': 'idade',
    'A003': 'sexo',
    'A004': 'cor',
    'A005': 'escolaridade',
    'A006A': 'ensino_publico_privado',
    'A006': 'frequentou_escola',
    'F001': 'tipo_moradia',
    'V1022': 'situacao_domiciliar',
    #Renda
    'C01011': 'rendimento',
    # Comportamento (lockdown)
    'B011': 'restringiu_contato_fisico',
    # Sintomas
    'B0011': 'febre',
    'B0012': 'tosse',
    'B0013': 'dor_garganta',
    'B0014': 'dificuldade_respirar',
    'B0015': 'dor_cabeca',
    'B0016': 'dor_peito',
    'B0017': 'nausea',
    'B0018': 'nariz_entupido',
    'B0019': 'fadiga',
    'B00110': 'dor_olhos',
    'B00111': 'perda_olfato_paladar',
    'B00112': 'dor_muscular',
    'B00113': 'diarreia',
    # Medico (Social)
    'B002':  'ida_unidade_saude',
    'B0043': 'hospital_publico',
    'B0046': 'hospital_privado',
    'B0041': 'posto_publico',
    'B0044': 'ambulatorio_privado',
    'B0042': 'pronto_socorro_publico',
    'B0045': 'pronto_socorro_privado',
    # Testes (covid)
    'B008': 'fez_teste',
    'B009A': 'teste_swab', # exame coletado com cotonete na boca e ou ou nariz
    'B009B': 'resultado_swab',
    'B009C': 'teste_sangue_dedo',
    'B009D': 'resultado_sangue_dedo',
    'B009E': 'teste_sangue_coleta',
    'B009F': 'resultado_sangue_coleta',
    # Comorbidade
    'B0101': 'diagnostico_diabetes',
    'B0102': 'diagnostico_hipertensao',
    'B0103': 'diagnostico_doenca_pulmao',
    'B0104': 'diagnostico_doenca_cardiaca',
    'B0105': 'diagnostico_depressao',
    'B0106': 'diagnostico_cancer',
    #Afastamento (trabalho)
    'C002': 'afastado_trabalho',
    'C003': 'motivo_do_afastamento',
    'C004': 'remunerado_continua',
    # trabalho
    'C006': 'mais_de_um_trabalho',
    'C007': 'tipo_trabalho',
    'C008': 'horas_trabalho_normal',
    'C013': 'trabalho_remoto_semana_passada'
})

df_covid.head()

,uf,idade,sexo,cor,escolaridade,ensino_publico_privado,frequentou_escola,tipo_moradia,situacao_domiciliar,rendimento,...,diagnostico_doenca_cardiaca,diagnostico_depressao,diagnostico_cancer,afastado_trabalho,motivo_do_afastamento,remunerado_continua,mais_de_um_trabalho,tipo_trabalho,horas_trabalho_normal,trabalho_remoto_semana_passada
0,rondônia,36,homem,parda,médio completo,não aplicavel,não aplicavel,próprio - já pago,urbana,801 - 1.600,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não,empregado do setor privado,48.0,não aplicavel
1,rondônia,30,mulher,parda,superior completo,não aplicavel,não aplicavel,próprio - já pago,urbana,801 - 1.600,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não,conta propria,36.0,não aplicavel
2,rondônia,13,homem,parda,fundamental incompleto,não aplicavel,sim,próprio - já pago,urbana,não aplicável,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel
3,rondônia,11,homem,parda,fundamental incompleto,não aplicavel,sim,próprio - já pago,urbana,não aplicável,...,não,não,não,não aplicavel,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel
4,rondônia,57,mulher,branca,fundamental incompleto,não aplicavel,não aplicavel,próprio - já pago,urbana,não aplicável,...,não,não,não,não,não aplicável,não aplicavel,não aplicavel,não aplicável,0.0,não aplicavel


## Exporta dados tratados

In [66]:
df_covid.to_csv('/content/drive/MyDrive/PosTech_Analise_de_dados/FASE3/tech_challenge_3/dados/df_covid.csv', index = False)